# 사용자 정의 모델 클래스 
- 부모 클래스 : nn.Module
- 필수 오버라이딩 
    - _ _init_ _() : 모델층 구성 즉, 설계
    - forward() : 순방향 학습 진행 코드 구현
- 동적 모델 
    - container 모듈 중 nn.ModuleList() 사용해서 동적으로 Layer 추가 
        - forward 기능 미 제공 
        - layer 인스턴스 요소 사이에 연관성 없음 
        - layer 인스턴스 요소는 인덱싱으로 접근 

In [107]:
import torch                                # 텐서 관련 모듈 
import torch.nn as nn                       # 인공신경망 관련 모듈 
import torch.nn.functional as F             # 인공신경망 관련 함수들 모듈 (손실함수, 활성화 함수 등등)
import torch.optim as optim                 # 최적화 관련 모듈 (가중치, 절편 빠르게 찾아주는 알고리즘 )
from torchinfo import summary               # 모델 구조 및 정보 관련 모듈 
from torchmetrics.regression import *       # 회귀 성능 지표 관련 모듈 
from torchmetrics.classification import *   # 분류 성능 지표 관련 모듈 

- [기본] 신경망 클래스 <hr>
    - 입력층 - 입력 : 피쳐수 고정 
    - 출력층 - 출력 : 타겟수 고정 
    - 은닉층 - 고정     

In [108]:
# 랜덤 고정 
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정 
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

- 모델 설계 - 동적 모델 <hr>
    - 목표 : 은닉층의 개수가 동적인 모델 
    - 조건 
        - 입력층과 출력층 개수 동적 => 입력층의 입력값, 출력값의 출력값 
        - 은닉층의 개수 동적 + 퍼셉트론 개수 고정 => 은닉층의 개수, 은닉층 마다 퍼셉트론 수

In [109]:
# 모델이름 : DynamicModel
# 부모 클래스 : nn.Module
# 매개변수 : in_in, out_out, h_ins = [], n_outs = []

In [110]:
class DynamicModel(nn.Module):


    # 모델 구조 설계 함수 즉, 생성자 메서드 
    def __init__(self, in_in,in_out, out_out , h_ins = [], h_outs = []):
        super().__init__()

        self.in_layer = nn.Linear(in_in, h_ins[0] if len(h_ins) else in_out)
        
        self.h_layers =nn.ModuleList()
        for idx in range(len(h_ins)):
            self.h_layers.append(nn.Linear(h_ins[idx],h_outs[idx]))
        
        self.out_layer = nn.Linear(h_outs[-1] if len(h_outs) else in_out, out_out)


    # 학습 진행 콜백 메서드 
    def forward(self, x):
        # 입력층 
        y = self.in_layer(x)    # y = x1w1 + x2w2 + x3w3 + b
        y = F.relu(y)           # 결과값은 0 ~ 
        # y = F.relu(self.in_layer(x)) 로 줄여쓸 수 ㅇ 

        # 은닉층 
        for linear in self.h_layers:
            y = linear(y)
            y = F.relu(y)
            # y = F.relu(linear(y)) 로 줄여쓸 수 ㅇ 

        # 출력층  
        return self.out_layer(y)


In [111]:
class DynamicModel_h(nn.Module):


    # 모델 구조 설계 함수 즉, 생성자 메서드 
    def __init__(self, in_in,in_out, out_in, out_out , h_ins = [], h_outs = []):
        super().__init__()

        self.in_layer = nn.Linear(in_in, h_ins[0] if len(h_ins) else in_out)
        # self.h_layers =nn.ModuleList( [nn.Linear(h_inout, h_inout) for _ in range(h_cnt)])
        self.out_layer = nn.Linear(h_outs[-1] if len(h_outs) else out_in, out_out)


    # 학습 진행 콜백 메서드 
    def forward(self, x):
        # 입력층 
        y = self.in_layer(x)    # y = x1w1 + x2w2 + x3w3 + b
        y = F.relu(y)           # 결과값은 0 ~ 
        # y = F.relu(self.in_layer(x)) 로 줄여쓸 수 ㅇ 

        # 은닉층 
        for linear in self.h_layers:
            y = linear(y)
            y = F.relu(y)
            # y = F.relu(linear(y)) 로 줄여쓸 수 ㅇ 

        # 출력층  
        return self.out_layer(y)


In [112]:
m1 = DynamicModel(3,5,2) # 은닉층이 없을 때 
print(m1)

DynamicModel(
  (in_layer): Linear(in_features=3, out_features=5, bias=True)
  (h_layers): ModuleList()
  (out_layer): Linear(in_features=5, out_features=2, bias=True)
)


In [113]:
m2 = DynamicModel_h(3,5,4,2) 
print(m2)

DynamicModel_h(
  (in_layer): Linear(in_features=3, out_features=5, bias=True)
  (out_layer): Linear(in_features=4, out_features=2, bias=True)
)


In [114]:
for name, param in m1.named_parameters():
    print(name, param.shape)

in_layer.weight torch.Size([5, 3])
in_layer.bias torch.Size([5])
out_layer.weight torch.Size([2, 5])
out_layer.bias torch.Size([2])


In [115]:
# 모델 인스턴스 생성 
h_ins , h_outs = [30,50,70],[50,70,30] # 은닉층 3개 
#               = range(10,101,20), range(30,121,20)
#               => 10 30 50 70 90 , 30 50 70 90 110
# h_inouts = [10, 30, 50, 70 ,90]
# => (10,30) (30,50), (50,70), (70,90)
# => out은 -1: 와야 함 -> 끝엔 없으므로 

m1 = DynamicModel(in_in=3, in_out=5, out_out=2, h_ins=h_ins, h_outs=h_outs)

In [116]:
print(m1)

DynamicModel(
  (in_layer): Linear(in_features=3, out_features=30, bias=True)
  (h_layers): ModuleList(
    (0): Linear(in_features=30, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=70, bias=True)
    (2): Linear(in_features=70, out_features=30, bias=True)
  )
  (out_layer): Linear(in_features=30, out_features=2, bias=True)
)


In [117]:
for name, param in m1.named_parameters():
    print(name, param.shape)

in_layer.weight torch.Size([30, 3])
in_layer.bias torch.Size([30])
h_layers.0.weight torch.Size([50, 30])
h_layers.0.bias torch.Size([50])
h_layers.1.weight torch.Size([70, 50])
h_layers.1.bias torch.Size([70])
h_layers.2.weight torch.Size([30, 70])
h_layers.2.bias torch.Size([30])
out_layer.weight torch.Size([2, 30])
out_layer.bias torch.Size([2])


학습 진행

In [118]:
# 임시 데이터 생성 
data_ts = torch.FloatTensor([[1,3,5],[2,4,6],[3,5,7],[4,6,8]])  # 2D 4x3
target = torch.FloatTensor([[7,9],[8,10],[9,11],[10,12]])  # 2D 4x2 필요 

In [119]:
# 모델 학습 
pre_y = m1(data_ts)
print(pre_y, pre_y.shape, sep='\n')

tensor([[-0.0087, -0.1601],
        [-0.0135, -0.1568],
        [-0.0182, -0.1557],
        [-0.0186, -0.1629]], grad_fn=<AddmmBackward0>)
torch.Size([4, 2])
